In [1]:
!pip install numpy==1.21.0 pandas==1.5.3 pyyaml pandasai tabulate

  Using cached numpy-1.21.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (2.0 kB)
  Using cached pandas-1.5.3-cp39-cp39-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached pandasai-2.2.15-py3-none-any.whl.metadata (10 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached duckdb-1.1.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (762 bytes)
  Using cached Faker-19.13.0-py3-none-any.whl.metadata (15 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached matplotlib-3.9.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached openai-1.50.2-py3-none-any.whl.metadata (24 kB)
  Using cached pillow-10.4.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using

In [83]:
# Import necessary libraries
import io
import re
import sys
import pandas as pd
import numpy as np 
from tabulate import tabulate
import matplotlib.pyplot as plt
from pandasai import Agent, SmartDataframe
# from azure_openai import AzureOpenAI
from pandasai.llm.openai import OpenAI
from openai import OpenAI

# Set up pandas options to display more rows and columns for better clarity
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [97]:
import io
import re
import sys
import os
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from openai import OpenAI
# from pandasai.llm.openai import OpenAI  # Uncomment if using pandasai's OpenAI wrapper

def generate_dataset_description(file_path, description=None, key_features=None):
    # Load the dataset
    df = pd.read_csv(file_path)
    
    # Prepare each part for the template
    columns = f"**Column Names:**\n\n- " + "\n- ".join(list(df.columns))
    head = f"**First 2 Rows of the DataFrame:**\n\n" + tabulate(df.head(2), headers='keys', tablefmt='pipe')
    
    buffer = io.StringIO()
    df.info(buf=buffer)
    info_str = buffer.getvalue()
    info = f"**DataFrame Info:**\n\n```\n{info_str}\n```"
    
    types = f"**Data Types:**\n\n" + tabulate(df.dtypes.reset_index(), headers=['Column', 'Data Type'], tablefmt='pipe')
    summary = f"**Summary Statistics:**\n\n" + tabulate(df.describe(), headers='keys', tablefmt='pipe')
    null_sum = "**Number of Nulls in Columns:**\n\n" + "\n".join([f"- **{col}**: {count}" for col, count in df.isnull().sum().items()])
    dupe_sum = f"**Number of Duplicate Rows:** {df.duplicated().sum()}"
    unique_sum = "**Number of Unique Values in Columns:**\n\n" + "\n".join([f"- **{col}**: {count}" for col, count in df.nunique().items()])
    
    # Set default values for optional arguments
    if description is None:
        description = "No description provided."
    
    if key_features is None:
        key_features = "No key features provided."
    
    # Format the output
    system = '''
    You are a data scientist tasked to explore a dataset and provide meaningful insights.
    To do this you will use the pandas Python library. Based on the questions you are asked, you will write Python code that should answer that question when you run it.

    Below are details about the dataset that will help you write the code. Consider the filename and column names when writing your response so the code works for this dataset.
    Do not make up column names, only use details that are provided.

    DATASET DETAILS
    ---
    FILENAME:
    {file_path}

    DESCRIPTION:
    {description}

    COLUMN NAMES:
    {columns}

    KEY COLUMNS:
    {key_features}

    HEAD:
    {head}

    SUMMARY:
    {summary}

    INFO:
    {info}

    NUMBER OF NULLS IN COLUMNS:
    {null_sum}

    NUMBER OF DUPLICATES IN COLUMNS:
    {dupe_sum}

    NUMBER OF UNIQUE VALUES IN COLUMNS:
    {unique_sum}

    If you are unable to write code for the given question, do the following:
       1. Explain what would be needed to answer this question.
       2. Give a suggestion on a better question to ask.
    '''
    
    # Insert values into the placeholders
    return system.format(
        file_path=file_path,
        description=description,
        columns=columns,
        key_features=key_features,
        head=head,
        summary=summary,
        info=info,
        null_sum=null_sum,
        dupe_sum=dupe_sum,
        unique_sum=unique_sum
    )

def execute_python_code(response):
    # Regex to find code wrapped in ```python ``` blocks
    code_match = re.search(r'```python(.*?)```', response, re.DOTALL)
    
    if code_match:
        code = code_match.group(1).strip()  # Extract and clean the code
        
        # Capture the output of the code execution
        old_stdout = sys.stdout
        sys.stdout = io.StringIO()
        
        try:
            # Execute the code
            exec(code, globals())
            
            # Get the output from the code execution
            output = sys.stdout.getvalue()
            
            # Check if there are any plots generated
            plt.show()
            
            # Display the code, output, and any additional information
            display(Markdown(f"**Code Executed:**\n\n```python\n{code}\n```"))
            if output:
                display(Markdown(f"**Output:**\n\n```\n{output}\n```"))
        except Exception as e:
            # Handle any exceptions that occur during code execution
            output = f"Error executing code: {e}"
            display(Markdown(f"**Error:**\n\n```\n{output}\n```"))
        finally:
            # Restore standard output to its original state
            sys.stdout = old_stdout
    else:
        display(Markdown("**No Python code found in the response.**" + response))

def ask_question_and_display_output(question, file_path, description=None, key_features=None):
    # Generate system message with dataset details
    system_message = generate_dataset_description(file_path, description, key_features)
    
    # Initialize OpenAI (replace with your key and parameters)
    openai_client = OpenAI(
        organization=os.environ.get("OPENAI_ORG"),
        project=os.environ.get("OPENAI_PRJ"),
        )


    # Run the OpenAI completion
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question}
        ],
        max_tokens=500
    )

    # Extract the response content
    response_content = response.choices[0].message.content
    
    # Display the question and AI-generated response
    display(Markdown(f"**Question:** {question}"))
    display(Markdown(f"**AI Response:**\n\n{response_content}"))
    
    # Execute the Python code within the response
    execute_python_code(response_content)

# Example usage
file_path = 'electronic_sales_data.csv'
description = "This dataset contains sales transaction records for an electronics company."
key_features = """
Customer ID: Unique identifier for each customer.
Age: Age of the customer (numeric)
Gender: Gender of the customer (Male or Female)
Loyalty Member: (Yes/No) (Values change by time, so pay attention to who cancelled and who signed up)
Product Type: Type of electronic product sold (e.g., Smartphone, Laptop, Tablet)
SKU: a unique code for each product.
Rating: Customer rating of the product (1-5 stars) (Should have no Null Ratings)
Order Status: Status of the order (Completed, Cancelled)
Payment Method: Method used for payment (e.g., Cash, Credit Card, Paypal)
Total Price: Total price of the transaction (numeric)
Unit Price: Price per unit of the product (numeric)
Quantity: Number of units purchased (numeric)
Purchase Date: Date of the purchase (format: YYYY-MM-DD)
Shipping Type: Type of shipping chosen (e.g., Standard, Overnight, Express)
Add-ons Purchased: List of any additional items purchased (e.g., Accessories, Extended Warranty)
Add-on Total: Total price of add-ons purchased (numeric)
"""
question = "What is the average sales amount for each month?"
ask_question_and_display_output(question, file_path, description, key_features)


**Question:** What is the average sales amount for each month?

**AI Response:**

To calculate the average sales amount for each month, you'll first need to ensure that the `Purchase Date` column is in datetime format. Then, you can group the data by month and calculate the average `Total Price`.

Here is the code to achieve this:

```python
import pandas as pd

# Load the dataset
df = pd.read_csv('electronic_sales_data.csv')

# Convert the 'Purchase Date' column to datetime format
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])

# Add a new column 'Month' that represents the month of the purchase
df['Month'] = df['Purchase Date'].dt.to_period('M')

# Group by 'Month' and calculate the average 'Total Price'
average_sales_per_month = df.groupby('Month')['Total Price'].mean()

# Print the result
print(average_sales_per_month)
```

This code will output the average sales amount for each month.

**Code Executed:**

```python
import pandas as pd

# Load the dataset
df = pd.read_csv('electronic_sales_data.csv')

# Convert the 'Purchase Date' column to datetime format
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])

# Add a new column 'Month' that represents the month of the purchase
df['Month'] = df['Purchase Date'].dt.to_period('M')

# Group by 'Month' and calculate the average 'Total Price'
average_sales_per_month = df.groupby('Month')['Total Price'].mean()

# Print the result
print(average_sales_per_month)
```

**Output:**

```
Month
2023-09    2536.641000
2023-10    2544.968551
2023-11    2595.274956
2023-12    2475.875413
2024-01    3230.928487
2024-02    3153.848218
2024-03    3231.664711
2024-04    3322.077443
2024-05    3275.899868
2024-06    3335.984787
2024-07    3285.635757
2024-08    3323.150946
2024-09    3376.468579
Freq: M, Name: Total Price, dtype: float64

```

In [81]:
# Load dataset
df = pd.read_csv('electronic_sales_data_cleaned.csv')

# Convert 'Purchase Date' to datetime
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])

# Extract year and month and create a new column
df['Year-Month'] = df['Purchase Date'].dt.to_period('M')

# Group by 'Year-Month' and calculate the mean of 'Total Price'
monthly_avg_sales = df.groupby('Year-Month')['Total Price'].mean().reset_index()

# Rename columns for clarity
monthly_avg_sales.columns = ['Year-Month', 'Average Sales Amount']

print(monthly_avg_sales)

   Year-Month  Average Sales Amount
0     2023-09           2536.641000
1     2023-10           2544.968551
2     2023-11           2595.274956
3     2023-12           2475.875413
4     2024-01           3230.928487
5     2024-02           3153.848218
6     2024-03           3231.664711
7     2024-04           3322.077443
8     2024-05           3275.899868
9     2024-06           3335.984787
10    2024-07           3285.635757
11    2024-08           3323.150946
12    2024-09           3376.468579


In [30]:
# Display the first few rows to inspect the dataframe's structure
print(df.head())


   Customer ID  Age Gender Loyalty Member Product Type      SKU  Rating  \
0         1000   53   Male             No   Smartphone  SKU1004       2   
1         1000   53   Male             No       Tablet  SKU1002       3   
2         1002   41   Male             No       Laptop  SKU1005       3   
3         1002   41   Male            Yes   Smartphone  SKU1004       2   
4         1003   75   Male            Yes   Smartphone  SKU1001       5   

  Order Status Payment Method  Total Price  Unit Price  Quantity  \
0    Cancelled    Credit Card      5538.33      791.19         7   
1    Completed         Paypal       741.09      247.03         3   
2    Completed    Credit Card      1855.84      463.96         4   
3    Completed           Cash      3164.76      791.19         4   
4    Completed           Cash        41.50       20.75         2   

  Purchase Date Shipping Type              Add-ons Purchased  Add-on Total  
0    2024-03-20      Standard  Accessory,Accessory,Accessory   

In [31]:
# Display the dataframe's column data types
print(df.dtypes)


Customer ID            int64
Age                    int64
Gender                object
Loyalty Member        object
Product Type          object
SKU                   object
Rating                 int64
Order Status          object
Payment Method        object
Total Price          float64
Unit Price           float64
Quantity               int64
Purchase Date         object
Shipping Type         object
Add-ons Purchased     object
Add-on Total         float64
dtype: object


In [32]:
# Replace 'date' with the name of your date column
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'], errors='coerce')

# Verify the conversion
print(df.dtypes)


Customer ID                   int64
Age                           int64
Gender                       object
Loyalty Member               object
Product Type                 object
SKU                          object
Rating                        int64
Order Status                 object
Payment Method               object
Total Price                 float64
Unit Price                  float64
Quantity                      int64
Purchase Date        datetime64[ns]
Shipping Type                object
Add-ons Purchased            object
Add-on Total                float64
dtype: object


In [33]:
# Identify rows where the date conversion failed
invalid_dates = df[df['Purchase Date'].isna()]
print(invalid_dates)


Empty DataFrame
Columns: [Customer ID, Age, Gender, Loyalty Member, Product Type, SKU, Rating, Order Status, Payment Method, Total Price, Unit Price, Quantity, Purchase Date, Shipping Type, Add-ons Purchased, Add-on Total]
Index: []


In [37]:
df.to_csv('electronic_sales_data_cleaned.csv', index=False)

In [39]:
from pandasai import SmartDataframe
from pandasai.llm import OpenAI
from pandasai.helpers.openai_info import get_openai_callback

llm = OpenAI(api_token=os.environ.get("OPENAI_API_KEY"), model=model)
ai_df = SmartDataframe("electronic_sales_data_cleaned.csv", config={"llm": llm})

try:
    with get_openai_callback() as cb:
        response = ai_df.chat("Calculate the total sales amount.")
        print(response)
        print(cb)
except Exception as e:
    print(f"An error occurred: {e}")


42629615.57
Tokens Used: 613
	Prompt Tokens: 427
	Completion Tokens: 186
Total Cost (USD): $ 0.004925


### 1. General Data Structure and Quality

In [ ]:
# Display information about the dataset, including column names, data types, and non-null counts
df.info()

# Check for missing values in each column
missing_values = df.isnull().sum()
print("Columns with missing values:")
print(missing_values[missing_values > 0])

# Check for duplicate entries
duplicates_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates_count}")

### 2. Overview of Sales Performance


In [ ]:
# Calculate total sales revenue (assuming 'Sales_Amount' is the relevant column)
total_sales = df['Sales_Amount'].sum()
print(f"Total Sales Revenue: ${total_sales:.2f}")

# Convert 'Date' column to datetime format for further analysis
df['Date'] = pd.to_datetime(df['Date'])

# Group by month and calculate total sales
monthly_sales = df.resample('M', on='Date')['Sales_Amount'].sum()
print("Monthly Sales Trend:")
print(monthly_sales)

# Plot the monthly sales trend
plt.figure(figsize=(10, 5))
monthly_sales.plot()
plt.title('Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.show()


### 3. Customer Behavior Insights


In [ ]:
# Find the number of unique customers
unique_customers = df['Customer_ID'].nunique()
print(f"Total Unique Customers: {unique_customers}")

# Calculate the frequency of purchases per customer
purchase_frequency = df['Customer_ID'].value_counts()
print("Top 5 most frequent buyers:")
print(purchase_frequency.head(5))

# Calculate the average time interval between purchases for each customer
customer_avg_interval = df.groupby('Customer_ID')['Date'].apply(lambda x: x.sort_values().diff().mean())
print("Average purchase intervals (in days) for the top 5 customers:")
print(customer_avg_interval.head(5))


### 4. Product and Category Analysis


In [ ]:
# Group data by 'Product_ID' to find top-selling products by quantity
top_selling_products = df.groupby('Product_ID')['Quantity'].sum().sort_values(ascending=False)
print("Top 5 selling products by quantity:")
print(top_selling_products.head(5))


### 5. Sales Trends and Patterns


In [ ]:
# Group data by 'Region' and sum 'Sales_Amount' (assuming 'Region' is a column)
sales_by_region = df.groupby('Region')['Sales_Amount'].sum().sort_values(ascending=False)
print("Total sales by region:")
print(sales_by_region)



### 6. Pricing and Discounts


In [ ]:
# Calculate correlation between 'Product_Price' and 'Quantity'
correlation = df['Product_Price'].corr(df['Quantity'])
print(f"Correlation between product price and quantity sold: {correlation:.2f}")



### 7. Potential Issues and Data Cleaning


In [ ]:
# Identify outliers in 'Sales_Amount' using the IQR method
Q1 = df['Sales_Amount'].quantile(0.25)
Q3 = df['Sales_Amount'].quantile(0.75)
IQR = Q3 - Q1

outliers = df[(df['Sales_Amount'] < (Q1 - 1.5 * IQR)) | (df['Sales_Amount'] > (Q3 + 1.5 * IQR))]
print(f"Number of outliers in Sales Amount: {outliers.shape[0]}")



### 8. Customer Segmentation and Retention


In [ ]:
# Create a simple customer segmentation based on purchase frequency
df['Purchase_Frequency'] = df['Customer_ID'].map(purchase_frequency)
bins = [0, 1, 5, 10, 50, df['Purchase_Frequency'].max()]
labels = ['One-time', 'Occasional', 'Frequent', 'Loyal', 'Super Loyal']
df['Customer_Segment'] = pd.cut(df['Purchase_Frequency'], bins=bins, labels=labels)
segment_counts = df['Customer_Segment'].value_counts()
print("Customer segments based on purchase frequency:")
print(segment_counts)



### 9. Automated Insights with pandas ai and Azure OpenAI


In [ ]:
# Set up Azure OpenAI for pandas_ai (replace with your own API key and endpoint)
openai = AzureOpenAI(api_key='YOUR_API_KEY', endpoint='YOUR_API_ENDPOINT')
pandas_ai = PandasAI(llm=openai)

# Use pandas_ai to interactively query the data
response = pandas_ai(df, "What is the average sales amount for each month?")
print("Pandas AI Response:")
print(response)
